In [1]:
# Based on https://viso.ai/deep-learning/what-are-liquid-neural-networks/

import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
def initialize_weights(input_dim, reservoir_dim, output_dim, spectral_radius):
    # initialize input weights randomly
    reservoir_weights = np.random.randn(reservoir_dim, reservoir_dim)
    # scale reservoir weights to achieve desired spectral radius
    reservoir_weights *= spectral_radius / np.max(np.abs(np.linalg.eigvals(reservoir_weights)))
    # initialize input-to-reservoir weights randomly
    input_weights = np.random.randn(reservoir_dim, input_dim)
    # initialize output weights to zero
    output_weights = np.zeros((reservoir_dim, output_dim))
    
    return reservoir_weights, input_weights, output_weights

In [3]:
def train_lnn(input_data, labels, reservoir_weights, input_weights, output_weights, leak_rate, num_epochs):
    num_samples = input_data.shape[0]
    reservoir_dim = reservoir_weights.shape[0]
    reservoir_states = np.zeros((num_samples, reservoir_dim))
    
    for epoch in range(num_epochs):
        for i in range(num_samples):
            # update reservoir state
            if i > 0:
                reservoir_states[i, :] = (1 - leak_rate) * reservoir_states[i - 1, :]
            reservoir_states[i, :] += leak_rate * np.tanh(
                np.dot(reservoir_weights, reservoir_states[i, :]) + 
                np.dot(input_weights, input_data[i, :]))
            
        # train output weights
        output_weights = np.dot(np.linalg.pinv(reservoir_states), labels)
        # compute training accuracy
        train_prediction = np.dot(reservoir_states, output_weights)
        train_accuracy = np.mean(np.argmax(train_prediction, axis=1) == np.argmax(labels, axis=1))
        print(f'Epoch {epoch + 1}/{num_epochs}, training accuracy: {train_accuracy:.4f}')
        
    return output_weights

In [4]:
def predict_lnn(input_data, reservoir_weights, input_weights, output_weights, leak_rate):
    num_samples = input_data.shape[0]
    reservoir_dim = reservoir_weights.shape[0]
    reservoir_states = np.zeros((num_samples, reservoir_dim))
    
    for i in range(num_samples):
        # update reservoir state
        if i > 0:
            reservoir_states[i, :] = (1 - leak_rate) * reservoir_states[i - 1, :]
        reservoir_states[i, :] += leak_rate * np.tanh(
            np.dot(reservoir_weights, reservoir_states[i, :]) + 
            np.dot(input_weights, input_data[i, :]))
        
    # compute predictions using output weights
    predictions = np.dot(reservoir_states, output_weights)
    return predictions

In [5]:
# set LNN hyperparameters
input_dim = 784
reservoir_dim = 1_000
output_dim = 10
leak_rate = 0.1
spectral_radius = 0.9
num_epochs = 10
input_scale = 1.0 / 255.0

In [6]:
# load and preprocess MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
x_train = x_train.reshape(x_train.shape[0], input_dim) * input_scale
x_test = x_test.reshape(x_test.shape[0], input_dim) * input_scale

In [7]:
# initialize LNN weights
reservoir_weights, input_weights, output_weights = initialize_weights(
    input_dim, reservoir_dim, output_dim, spectral_radius)

In [ ]:
# train LNN
output_weights = train_lnn(
    x_train, y_train, reservoir_weights, input_weights, output_weights, leak_rate, num_epochs)

In [ ]:
# evaluate the LNN on test data
predictions = predict_lnn(
    x_test, reservoir_weights, input_weights, output_weights, leak_rate)
test_accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))
print(f'Test accuracy: {test_accuracy:.4f}')

To tune the hyperparameters you've listed for your Liquid Neural Network (LNN), you can follow a systematic approach using a combination of search strategies. Here's how you can proceed with the given parameters and some additional ones you might consider:

### Existing Hyperparameters:
1. **input_dim**: This is typically determined by the feature size of your dataset and usually does not require tuning.
2. **reservoir_dim**: The number of neurons in the reservoir. This can significantly affect the capacity and memory of the network.
3. **output_dim**: Similar to input_dim, this is usually determined by the number of classes or the output size required by your task.
4. **leak_rate**: Controls the rate at which the reservoir's internal state updates. It's crucial for the dynamics of the network.
5. **spectral_radius**: Influences the echo state property of the reservoir. It's vital for the stability and performance of the network.
6. **num_epochs**: The number of times the entire training dataset is passed forward and backward through the LNN.

### Additional Hyperparameters to Consider:
7. **input_scaling**: The factor by which input signals are scaled before being fed into the reservoir. This can affect how the inputs influence the reservoir dynamics.
8. **connectivity**: The proportion of nonzero connections in the reservoir. Sparse connections often lead to more efficient and diverse dynamic reservoirs.
9. **regularization_parameter**: Helps prevent overfitting by adding a penalty to the loss function based on the weights' magnitude.

### Tuning Strategy:
1. **Define a Performance Metric**: Choose a metric like accuracy, mean squared error, etc., depending on your specific task.

2. **Data Splitting**: Split your data into training, validation, and test sets.

3. **Search Strategy**:
   - **Grid Search**: Start with a coarse grid search over a broad range of values for `reservoir_dim`, `leak_rate`, `spectral_radius`, `input_scaling`, and `connectivity`.
   - **Random Search**: Conduct a random search within specific ranges based on the results from the grid search to find more optimal values.
   - **Bayesian Optimization**: Use Bayesian optimization for fine-tuning the most sensitive parameters, typically `leak_rate` and `spectral_radius`.

4. **Evaluation and Iteration**:
   - For each combination of hyperparameters, train the LNN on the training set and evaluate it on the validation set using your chosen performance metric.
   - Record the performance for each set of parameters.

5. **Refinement**:
   - Narrow down the ranges for the hyperparameters based on the results from previous steps and repeat the search if necessary.

6. **Final Evaluation**:
   - Train the LNN using the best hyperparameters from the validation phase on the combined training and validation set.
   - Evaluate the model on the test set to gauge its performance on unseen data.

7. **Analysis**:
   - Analyze how different hyperparameters impacted the performance and stability of the LNN.
   - Adjust the tuning strategy based on these insights if needed.

By following this structured approach, you can effectively tune the hyperparameters of your Liquid Neural Network to optimize its performance for your specific task. Remember, tuning is an iterative process, and multiple rounds might be necessary to find the optimal configuration.